In [3]:
words = open('names.txt').read().splitlines()

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [5]:
import torch

In [11]:
block_size = 3
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size

    for c in w + '.':
        ix = stoi[c]
        X.append(context)
        Y.append(ix)

        print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [7]:
C = torch.randn((27, 2))

In [14]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [ ]:
W1 = torch.randn((6, 100))
B1 = torch.randn(100)

In [16]:
h = emb.view(emb.shape[0], 6) @ W1 + B1

tensor([[-0.9341,  0.1185],
        [-0.9341,  0.1185],
        [-0.9341,  0.1185],
        [ 0.0935, -1.8797],
        [ 0.3966, -1.0896],
        [-0.9341,  0.1185],
        [-0.9341,  0.1185],
        [-0.9341,  0.1185],
        [ 0.4288,  0.3329],
        [ 0.0035, -0.8770],
        [ 0.6129,  0.4535],
        [ 0.2810,  0.0829],
        [-0.9341,  0.1185],
        [-0.9341,  0.1185],
        [-0.9341,  0.1185],
        [-0.3225, -0.8793],
        [-0.9341,  0.1185],
        [-0.9341,  0.1185],
        [-0.9341,  0.1185],
        [ 0.6129,  0.4535],
        [ 1.6991, -0.8750],
        [-0.3225, -0.8793],
        [ 0.9626, -1.2810],
        [ 0.0935, -1.8797],
        [ 0.0035, -0.8770],
        [-0.9341,  0.1185],
        [-0.9341,  0.1185],
        [-0.9341,  0.1185],
        [ 1.6991, -0.8750],
        [ 0.4288,  0.3329],
        [-0.2925,  1.0948],
        [ 0.1302,  0.0990]])

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17]])